In [1]:
!apt-get update

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!java -version

!pip install pyspark


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [14]:
#Apache Spark : Genel amaçlı paralel veri işleme frameworku

#Hadoop üzerinde çalışabilir

#Büyük Veri İşleme, Büyük Veri ile Makine Öğrenmesi ===> HADOOP + Spark



from pyspark.sql.functions import *
from pyspark.sql.functions import UserDefinedFunction

from pyspark.sql.types import StringType,IntegerType,DateType,DoubleType

from pyspark.sql import SparkSession #DataFrame objeleri yaratmak için gerekli

from pyspark.sql.types import StructType,StructField


spark = SparkSession.builder\
.appName("Spark Dataframe Intro")\
          .getOrCreate()

#Spark SQL

movies = spark.read\
              .option("inferSchema","True")\
              .option("header","true")\
              .csv("movies.csv")
movies.printSchema()                
movies.createOrReplaceTempView("world")



ratings = spark.read\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .csv("ratings.csv")              

ratings.createOrReplaceTempView("ratings")
time_stamp_to_date_for_ratings = spark.sql("select userId, movieId, rating, date_sub(from_unixtime(cast(timestamp/1 as bigint)),0) as timestamp from ratings").createOrReplaceTempView("ratings")





ratings.printSchema() 



tags = spark.read\
      .option("inferSchema", "true")\
      .option("header", "true")\
      .csv("tags.csv")

tags.printSchema()

tags.createOrReplaceTempView("tags")

time_stamp_to_date_for_ratings = spark.sql("select userId, movieId, tag, date_sub(from_unixtime(cast(timestamp/1 as bigint)),0) as  timestamp from tags").createOrReplaceTempView("tags")



#a = spark.sql("select count(distinct tag) from tags").show()

#Firstly we need to convert timestamp value to date. we need to use  date_sub(from_unixtime(cast(timestamp/1 as bigint)),0) 

res = spark.sql("""select tag,round(avg(rating),2) as average_ratings  from world a 
left join ratings b on a.movieId = b.movieId 
left join tags c  on b.userID = c.userID and b.timestamp = c.timestamp
group by tag 
order by average_ratings asc
""").show(5000)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: integer (nullable = true)

+--------------------+---------------+
|                 tag|average_ratings|
+--------------------+---------------+
|             passion|           0.75|
|        way too long|            1.0|
|  Visually appealing|           2.25|
|          melancholy|           2.25|
|        Motivational|           2.25|
|           brilliant|           2.25|
|              lovely|           2.25|
|               anger|           2.25|
|            colorful|           2.25|
|     quirky romantic|           2.25|
|         """artsy"""|           2.25|
|     